# Introduction

Currently, rotation matching is done psuedo-stochastically. Here we frame rotation matching as a minimum cost-problem and propose an adaptable algorithm built on a linear sum optimizer method to non-randomally assign students to rotations based on a ranked preference matrix.

# Non-random Rotation Matching Algorithm

3rd year rotations at CCF include IM/Surgery, Peds/Ob/Gyn, Neuro/Psych, or Ambulatory for roughly 75 students. These are assigned the indicies 0-3, respectively.

In [1]:
import numpy as np
import pandas as pd
# using optimized variant of the Hungarian Method to solve linear sums problem
from scipy.optimize import linear_sum_assignment
np.random.seed(44106)


rotationdict = {
  0: "IM/Surgery",
  1: "Peds/Ob/Gyn",
  2: "Neuro/Psych",
  3: "Ambulatory"
}

# this will be replaced by reading in a preference matrix based on user submissions
cost_matrix = np.random.randint(4, size = (75, 4))

# cost_df = pd.read_csv("./preferences.csv")
# studentIDs = cost_df['studentID']
# cost_matrix = pd.DataFrame.to_numpy(cost_df.drop(columns=['studentID']))

Linear sum assignment problems require a square matrix; to solve this problem, we propose tiling the ranked preference matrix, which has the added benefit of ensuring an equal distribution of students in each block.

In [2]:
# pad matrix to multiples of 4 :: add one ghost row and ceil(75/4) - 1 duplicate columns
cost = cost_matrix.copy()

phantom_students = 0
# add phantom rows of all equal until there are a multiple of 4 rows
while(np.shape(cost)[0] % 4 != 0):
	cost = np.vstack([cost, [0, 0, 0, 0]])
	phantom_students += 1
	
# add duplicate columns to ensure square optimization problem 
cost = np.tile(cost, (1, np.shape(cost)[0] // 4))
print(np.shape(cost)) # should be a square matrix at this point

(76, 76)


We can also add a scaled penalty term to each choice. This is particularly useful when optimizing to avoid last-choice rotation. The initial penalty term is linear, but an exponent will skew away from last-preference rotation. 

In [3]:
cost = np.power(cost, 5) # 5th power penalty

The core of the algorithm involves solving a series of minimal cost optimization problems for each rotation placement. `rotation_calc()` calculates the optimal assignment of a given rotation via the Hungarian Algorithm, after which `update_cost_matrix()` is called to "remove" the rotations that are used up for a given student (row). Since the Hungarian Algorithm and its variants rely on a square matrix as input, we use an artificially inflated penalty that approaches infinity in the limit (arbitrarily large) to ensure that a given student is not assigned to the same rotation twice rather than deleting the rotation from the student's preference. This part of the algorithm can be further improved and adapted for various use cases. 

In [4]:
# run linear_sum_assignment on cost matrix --> optimal results are stored in col_index
def rotation_calc():
	row_ind, col_ind = linear_sum_assignment(cost) 
	err = cost[row_ind, col_ind].sum() # the "cost" in this case is the total deviation from everyone getting their first preference
	rotation_index = col_ind % 4 # re-wraps the indicies to their human readable form 
	rotations = [rotationdict.get(index) for index in rotation_index]
	update_cost_matrix(row_ind, rotation_index)
	return rotations, err

# greatly increases the penalty of rematching to the same rotation
def update_cost_matrix(row_ind, col_ind):
	for i in range(len(col_ind)):
		for mul in range(76//4):
			cost[i][(4 * mul) + col_ind[i]] = 1000

We invoke the algorithm 4 times: once for each rotation. The `_rotation` variable stores the optimal rotation assignment (mapped back to its plain text counterpart), while the `_rotation_err` variable stores the deviation from the optimal solution. For example, if one student got their 3rd choice rotation while everyone else got their first choice rotation, the total error rotation error would be 2 (0 + 0 + ... + 0 + 2). 

In [5]:
first_rotation, first_rotation_err = rotation_calc()

# rerun linear_sum_assignment on new matrix for second rotation preference
second_rotation, second_rotation_err = rotation_calc()

# rerun linear_sum_assignment on new matrix for third rotation preference
third_rotation, third_rotation_err = rotation_calc()

# rerun linear_sum_assignment on new matrix for fourth rotation preference
fourth_rotation, fourth_rotation_err = rotation_calc()

# Performance and Validation

In [6]:
import pandas as pd

df_dict = {
    "first_rotation": first_rotation, 
    "second_rotation": second_rotation, 
    "third_rotation": third_rotation, 
    "fourth_rotation": fourth_rotation
    }

rotations = pd.DataFrame(df_dict)
rotations.drop(rotations.tail(phantom_students).index, inplace = True) # remove the phantom students

rotations

,first_rotation,second_rotation,third_rotation,fourth_rotation
0,Neuro/Psych,Peds/Ob/Gyn,IM/Surgery,Ambulatory
1,Neuro/Psych,IM/Surgery,Peds/Ob/Gyn,Ambulatory
2,Neuro/Psych,IM/Surgery,Ambulatory,Peds/Ob/Gyn
3,Neuro/Psych,Peds/Ob/Gyn,Ambulatory,IM/Surgery
4,Peds/Ob/Gyn,Neuro/Psych,IM/Surgery,Ambulatory
...,...,...,...,...
70,Neuro/Psych,Peds/Ob/Gyn,Ambulatory,IM/Surgery
71,Neuro/Psych,Ambulatory,IM/Surgery,Peds/Ob/Gyn
72,IM/Surgery,Peds/Ob/Gyn,Ambulatory,Neuro/Psych
73,IM/Surgery,Neuro/Psych,Ambulatory,Peds/Ob/Gyn


In [7]:
# this block makes sure that no student is double assigned a rotation
sum(rotations['first_rotation'] == rotations['second_rotation']) +\
sum(rotations['first_rotation'] == rotations['third_rotation']) +\
sum(rotations['first_rotation'] == rotations['fourth_rotation']) +\
sum(rotations['second_rotation'] == rotations['third_rotation']) +\
sum(rotations['second_rotation'] == rotations['fourth_rotation']) +\
sum(rotations['third_rotation'] == rotations['fourth_rotation'])

0

In [8]:
reverse_rotation_dict = {v: k for k, v in rotationdict.items()}
rotation_index = rotations.replace({
    "first_rotation": reverse_rotation_dict, 
    "second_rotation": reverse_rotation_dict,
    "third_rotation": reverse_rotation_dict,
    "fourth_rotation": reverse_rotation_dict})
preference_df = pd.DataFrame(cost_matrix).rename(
    {0: "first_choice", 
    1: "second_choice", 
    2: "third_choice", 
    3: "fourth_choice"}, axis = 1)

In [24]:
IM_ranked_first = (preference_df['first_choice'] == 0)
IM_rotation_first = (rotation_index['first_rotation'] == 0)
print(f'Students who ranked and recieved IM/Surgery as their first choice: ', sum(IM_ranked_first & IM_rotation_first))

IM_ranked_first = (preference_df['first_choice'] == 1)
IM_rotation_first = (rotation_index['first_rotation'] == 1)
print(f'Students who ranked and recieved Peds/Ob/Gyn as their first choice: ', sum(IM_ranked_first & IM_rotation_first))

IM_ranked_first = (preference_df['first_choice'] == 2)
IM_rotation_first = (rotation_index['first_rotation'] == 2)
print(f'Students who ranked and recieved Neuro/Psych as their first choice: ', sum(IM_ranked_first & IM_rotation_first))

IM_ranked_first = (preference_df['first_choice'] == 3)
IM_rotation_first = (rotation_index['first_rotation'] == 3)
print(f'Students who ranked and recieved Ambulatory as their first choice: ', sum(IM_ranked_first & IM_rotation_first))

Students who ranked and recieved IM/Surgery as their first choice:  13
Students who ranked and recieved Peds/Ob/Gyn as their first choice:  3
Students who ranked and recieved Neuro/Psych as their first choice:  7
Students who ranked and recieved Ambulatory as their first choice:  7


In [11]:
first_choice_first = sum(rotation_index['first_rotation'] == preference_df['first_choice'])
first_choice_second = sum(rotation_index['second_rotation'] == preference_df['first_choice'])
first_choice_last = sum(rotation_index['fourth_rotation'] == preference_df['first_choice'])
print(f'Students whose first choice rotation was assigned last: ', students_who_got_last_choice)
print(f'Students who had their first choice rotation first: ', students_who_got_first_choice)
print(f'Students who had their first choice rotation either first or second: ', students_who_got_first_choice + students_who_got_second_choice)

Students whose first choice rotation was assigned last:  10
Students who had their first choice rotation first:  30
Students who had their first choice rotation either first or second:  43


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=45536ed8-df8e-44fc-bf35-001ad3b54dcc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>